In [2]:
from langchain.llms import GPT4All
from langchain.embeddings import GPT4AllEmbeddings

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.vectorstores.faiss import FAISS
from embedding import general_embedding, update_embedding

from langchain import PromptTemplate, LLMChain
import os

In [3]:
# 定义大语言模型LLMs
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
gpt4all_path = './models/ggml-gpt4all-j-v1.3-groovy.bin' 
llm = GPT4All(model=gpt4all_path, callback_manager=callback_manager, verbose=True)

Found model file at  ./models/ggml-gpt4all-j-v1.3-groovy.bin
gptj_model_load: loading model from './models/ggml-gpt4all-j-v1.3-groovy.bin' - please wait ...
gptj_model_load: n_vocab = 50400
gptj_model_load: n_ctx   = 2048
gptj_model_load: n_embd  = 4096
gptj_model_load: n_head  = 16
gptj_model_load: n_layer = 28
gptj_model_load: n_rot   = 64
gptj_model_load: f16     = 2
gptj_model_load: ggml ctx size = 5401.45 MB
gptj_model_load: kv self size  =  896.00 MB
gptj_model_load: ................................... done
gptj_model_load: model size =  3609.38 MB / num tensors = 285


In [4]:
# 定义嵌入式模型embedding
# embeddings = GPT4AllEmbeddings()

from langchain.embeddings import LlamaCppEmbeddings
llama_path = './models/ggml-model-q4_0.bin' 
embeddings = LlamaCppEmbeddings(model_path=llama_path)

llama.cpp: loading model from ./models/ggml-model-q4_0.bin
llama.cpp: can't use mmap because tensors are not aligned; convert to new format to avoid this
llama_model_load_internal: format     = 'ggml' (old version with low tokenizer quality and no mmap support)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: ftype      = 2 (mostly Q4_0)
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size = 4113748.20 KB
llama_model_load_internal: mem required  = 5809.33 MB (+ 2052.00 MB per state)
...................................................................................................
.
llam

In [5]:
# 生成datastores
base_folder_path = './docs/base'
update_folder_path = './docs/update'
updata = True
if not os.path.exists("datastores"):
    general_embedding(base_folder_path)
elif os.listdir(update_folder_path):
    update_embedding(update_folder_path)

In [6]:
def similarity_search(query, index):
    matched_docs = index.similarity_search(query, k=3) 
    sources = []
    for doc in matched_docs:
        sources.append(
            {
                "page_content": doc.page_content,
                "metadata": doc.metadata,
            }
        )

    return matched_docs, sources

In [ ]:
# 加载datastores
index = FAISS.load_local("datastores", embeddings)

# 保持连续对话
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [7]:
# prompt 样例
template = """Please use the following context to answer questions.
Context: {context}
------------------------
Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# 定义链
llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)

# while True:
#     # query
#     question = input("Your question: ")

#     # 检索，得到上下文信息context
#     matched_docs, sources = similarity_search(question, index)
#     context = "\n".join([doc.page_content for doc in matched_docs])

#     llm_chain.prompt = llm_chain.prompt.partial(context=context)

#     # 将检索结果和问题一起输入LLM，输出结果
#     print(llm_chain.run(question))

In [8]:

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

from langchain.chains import ConversationalRetrievalChain
chat = ConversationalRetrievalChain.from_llm(llm, retriever=index.as_retriever(), memory=memory)

In [9]:
result = chat({"question": "What are the approaches to Task Decomposition?"})
result['answer']


llama_print_timings:        load time =   723.71 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =   989.71 ms /    11 tokens (   89.97 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =   995.34 ms


 The following is a list of different task decomposition methods used in industrial control systems, along with their advantages and disadvantages:
1. Hierarchical Task Analysis (HTA) - This method involves breaking down complex tasks into smaller sub-tasks that can be performed by individual workers or teams. It helps to identify bottlenecks and optimize the workflow for better performance. However, it may require a significant amount of time and resources to implement HTA in large industrial systems with many processes running simultaneously.
2. Workflow Management Systems (WMS) - These software tools help manage complex workflows by providing real-time visibility into tasks' progress, enabling workers to make adjustments as needed. WMS can be integrated within PLC applications for improved task management and performance optimization. However, they may require significant investment in hardware or licensing fees depending on the complexity of their features.
3. Task Assignment Algor

" The following is a list of different task decomposition methods used in industrial control systems, along with their advantages and disadvantages:\n1. Hierarchical Task Analysis (HTA) - This method involves breaking down complex tasks into smaller sub-tasks that can be performed by individual workers or teams. It helps to identify bottlenecks and optimize the workflow for better performance. However, it may require a significant amount of time and resources to implement HTA in large industrial systems with many processes running simultaneously.\n2. Workflow Management Systems (WMS) - These software tools help manage complex workflows by providing real-time visibility into tasks' progress, enabling workers to make adjustments as needed. WMS can be integrated within PLC applications for improved task management and performance optimization. However, they may require significant investment in hardware or licensing fees depending on the complexity of their features.\n3. Task Assignment A

In [10]:
result = chat({"question": "How does the Reflexion paper handle it?"})
result['answer']

 What is the main contribution of this paper to task decomposition and scheduling algorithms in industrial control systems, if any?


llama_print_timings:        load time =   723.71 ms
llama_print_timings:      sample time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings: prompt eval time =  2254.53 ms /    27 tokens (   83.50 ms per token)
llama_print_timings:        eval time =     0.00 ms /     1 runs   (    0.00 ms per run)
llama_print_timings:       total time =  2269.18 ms


 The main contribution of this paper on PLC programming for plant maintenance tasks is a detailed explanation of how to use specialized language tools like SFMPLS. It also provides an overview of the benefits of using these programs compared to traditional PC-based software and highlights some potential challenges in implementing them effectively, such as managing multiple languages or dealing with limited resources. Overall, this paper contributes valuable insights into PLC programming for plant maintenance tasks that can be useful for those working on industrial control systems who need a more efficient way to manage their workflows.

' The main contribution of this paper on PLC programming for plant maintenance tasks is a detailed explanation of how to use specialized language tools like SFMPLS. It also provides an overview of the benefits of using these programs compared to traditional PC-based software and highlights some potential challenges in implementing them effectively, such as managing multiple languages or dealing with limited resources. Overall, this paper contributes valuable insights into PLC programming for plant maintenance tasks that can be useful for those working on industrial control systems who need a more efficient way to manage their workflows.'

: 